# Домашнее задание к лекции "Основы веб-скрапинга"

## Задание 1

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

search_requests = ['python', 'анализ данных']

def scrape_habr(search_list):
    base_url = 'https://habr.com'
    params = '/ru/search/?q='
    rows = []
    for sreq in search_list:
        response = requests.get(base_url + params + sreq)
        b_data = BeautifulSoup(response.text)
        news = b_data.find_all('article', class_='tm-articles-list__item')
        for i in news:
            date = datetime.strptime(i.find('a', class_='tm-article-datetime-published tm-article-datetime-published_link').find('time').get('datetime'), '%Y-%m-%dT%H:%M:%S.%fZ')
            title = i.find('h2').find('a').find('span').text
            link = base_url + i.find('h2').find('a').get('href')
            rows.append({'utc_date': date, 'title': title, 'link': link})
    result = pd.DataFrame(rows)
    return result.drop_duplicates(subset=['link'], keep='first')

scrape_habr(search_requests).head()

,utc_date,title,link
0,2025-04-15 05:55:39,"Представлен открытый инструмент TARIFF, которы...",https://habr.com/ru/news/900892/
1,2021-12-13 06:00:03,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/companies/slurm/news/595093/
2,2022-01-20 15:37:16,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/companies/slurm/news/646825/
3,2023-09-19 04:42:15,Microsoft представила дополнение Python Editor...,https://habr.com/ru/news/761862/
4,2022-01-13 15:35:30,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/companies/slurm/news/645485/


## Задание 2

Функция из обязательной части задания должна быть расширена следующим образом:

*   кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
*   в датафрейме должны быть столбцы с полным текстом найденных материалов и рейтингом статьи:

<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <рейтинг>

In [ ]:
import time

def scrape_habr(search_list, pages):
    base_url = 'https://habr.com'
    param_1 = '/ru/search/page'
    param_2 = '/?q='
    all_links = []
    for sreq in search_list:
        for i in range(1, pages + 1):
            response = requests.get(base_url + param_1 + str(i) + param_2 + sreq)
            time.sleep(0.2)
            b_data = BeautifulSoup(response.text)
            news = b_data.find_all('article', class_='tm-articles-list__item')
            for j in news:
                link = base_url + j.find('h2').find('a').get('href')
                all_links.append(link)
    unique_links = set(all_links)
    rows = []
    for link in unique_links:
        b_info = BeautifulSoup(requests.get(link).text)
        time.sleep(0.2)
        date = datetime.strptime(b_info.find('span', class_='tm-article-datetime-published').find('time').get('datetime'), '%Y-%m-%dT%H:%M:%S.%fZ')
        title = b_info.find('h1').find('span').text
        text = b_info.select_one('div.article-formatted-body').get_text(' ', strip=True)
        # try: # Вариант для красивой разметки текста на абзацы и новые строки.
        #     text = b_info.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').text
        # except AttributeError:
        #     text = '\n'.join([x.get_text(' ', strip=True) for x in b_info.find_all('p')[:-1]])
        rating = b_info.find('div', class_='tm-votes-meter votes-switcher').find('span').text
        # try: # Вариант для парсинга корпоративного блога или старых публикаций без рейтинга
        #     rating = b_info.find('div', class_='tm-votes-meter').find('span').text
        # except AttributeError:
        #     rating = 'N/A'
        rows.append({'utc_date': date, 'title': title, 'link': link, 'text': text, 'rating': rating})
    return pd.DataFrame(rows)

scrape_habr(search_requests, 4)

,utc_date,title,link,text,rating
0,2023-08-22 17:14:29,Microsoft добавила Python в Excel,https://habr.com/ru/news/756266/,"22 августа 2023 года Microsoft сообщила , что ...",+22
1,2017-07-25 10:47:48,10 лет Школе анализа данных Яндекса,https://habr.com/ru/companies/yandex/articles/...,Сегодня исполняется 10 лет Школе анализа данны...,+53
2,2023-10-09 12:23:46,Три уровня погружения в Python. Запись докладо...,https://habr.com/ru/companies/selectel/news/76...,"Привет, Хабр! В сентябре мы провели Selectel P...",+2
3,2025-02-11 10:55:53,Автоматизация разведочного анализа данных (EDA...,https://habr.com/ru/companies/gazprombank/arti...,"Всем привет! Меня зовут Константин Некрасов, я...",+8
4,2025-01-21 09:46:27,В Steam вышла игра Joy of Programming — Softwa...,https://habr.com/ru/news/875278/,В Steam вышла игра под названием Joy of Progra...,+16
...,...,...,...,...,...
153,2024-05-09 04:43:47,Вышла первая бета-версия языка программировани...,https://habr.com/ru/news/813271/,8 мая 2024 года вышла первая бета-версия языка...,+8
154,2016-09-06 10:58:24,Онлайн-программа по анализу данных на Stepik.org,https://habr.com/ru/companies/stepic/articles/...,Осень 2016 года выдастся насыщенной: СПбАУ РАН...,+12
155,2025-06-17 10:20:42,Чему учат в онлайн-магистратуре Яндекса: специ...,https://habr.com/ru/companies/yandex_praktikum...,Яндекс и ИТМО открыли набор в онлайн-магистрат...,0
156,2014-07-07 05:03:44,Дайджест статей по анализу данных №4 (23.06.20...,https://habr.com/ru/articles/228895/,"Добрый день, уважаемые читатели. Пролетели 2 н...",+14
